<a href="https://colab.research.google.com/github/vidhibhamare/voice_emotion_spotify/blob/main/try1_voice_emotion_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tkinter import Tk, filedialog


In [ ]:
# Set the path to the dataset directory
dataset_dir = '/Users/annadadash/Desktop/voice-emotion/archive (4)'

# Define a dictionary to map emotion codes to emotion labels
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

# Create empty lists to store file paths and corresponding emotions
file_paths = []
emotions_list = []

# Walk through the dataset directory and get file paths and emotions
for subdir, dirs, files in os.walk(dataset_dir):
    for file in files:
        # Check if the file has a .wav extension
        if file.endswith('.wav'):
            # Get the file path
            file_path = os.path.join(subdir, file)
            # Get the file name
            file_name = os.path.basename(file_path)
            # Get the emotion label from the emotion code in the file name
            emotion = emotions[file_name.split('-')[2]]
            # Append the file path and emotion to the respective lists
            file_paths.append(file_path)
            emotions_list.append(emotion)

# Create a pandas DataFrame to store the file paths and emotions
ravdess_df = pd.DataFrame({'file_path': file_paths, 'emotion': emotions_list})

# Extract features using librosa
features = []
for index, row in ravdess_df.iterrows():
    # Load the audio file using librosa
    file_path = row['file_path']
    audio, sr = librosa.load(file_path)
    # Extract MFCCs (Mel Frequency Cepstral Coefficients) from the audio
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20)
    # Calculate the mean of the MFCCs for each audio file
    mfccs_mean = np.mean(mfccs, axis=1)
    # Append the mean MFCCs and the emotion label to the features list
    features.append(np.append(mfccs_mean, row['emotion']))

# Convert the features list to a pandas DataFrame
features_df = pd.DataFrame(features)
# Rename the last column to 'emotion'
features_df.rename(columns={20: 'emotion'}, inplace=True)

# Split the dataset into training and testing sets
X = features_df.iloc[:, :-1].values
y = features_df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Train an SVM (Support Vector Machine) model with RBF (Radial Basis Function) kernel
svm = SVC(kernel='rbf', C=10, gamma=0.1)
svm.fit(X_train, y_train)

# Predict the emotion labels for the test set using the trained SVM model
y_pred = svm.predict(X_test)

# Calculate the accuracy of the SVM model on the test set
accuracy = accuracy_score(y_test, y_pred)
print(f'Test set accuracy: {accuracy*100} %')


Test set accuracy: 95.65972222222221 %


In [ ]:
# import necessary libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import webbrowser
import tkinter as tk
from tkinter import filedialog, Label, Button, Frame
from PIL import ImageTk, Image

# set up Spotify API client credentials
client_id = '948c1190b4fa4f39a1dbd1c18ac7edb5'
client_secret = '5c9a3f25be054c418e4847de8f3a5ccf'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# create the main GUI window
root = tk.Tk()
root.title('Voice Emotion Detection')
root.geometry('794x500')


# function to get recommended songs based on a given mood
def get_recommendations(mood):
    # search for playlists with the given mood keyword
    results = sp.search(q='playlist:' + mood, type='playlist')
    # get the first playlist ID from the search results
    playlist_id = results['playlists']['items'][0]['id']
    # get the tracks from the playlist
    tracks = sp.playlist_tracks(playlist_id)
    # get the names of the first 5 tracks
    track_names = [track['track']['name'] for track in tracks['items'][:5]]
    return track_names

# function to open a song's Spotify URL in a web browser
def open_song_url(url):
    webbrowser.open(url)

# function to predict the emotion of a selected audio file and display recommended songs


import pygame.mixer

# function to predict the emotion of a selected audio file and display recommended songs
def predict_emotion():
    # ask the user to select an audio file
    file_path = filedialog.askopenfilename()
    # import the audio file and play it
    pygame.mixer.init()
    pygame.mixer.music.load(file_path)
    pygame.mixer.music.play()
    # extract MFCC features from the audio file using librosa
    audio, sr = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=20)
    mfccs_mean = np.mean(mfccs, axis=1)
    # transform the features using the StandardScaler
    X_new = sc.transform([mfccs_mean])
    # predict the emotion using the SVM model
    y_new = svm.predict(X_new)
    # display the predicted emotion and get recommended songs
    predicted_emotion.configure(text=f'Predicted emotion: {y_new[0]}\n \nTop 5 songs based on your emotion:')
    mood=y_new[0]
    recommended_songs = get_recommendations(mood)

    # destroy the previous recommended songs buttons
    for widget in recommended_songs_frame.winfo_children():
        widget.destroy()

    # display the recommended songs
    for i, song in enumerate(recommended_songs):
        button = tk.Button(recommended_songs_frame, text=song, command=lambda url=sp.search(q=song, type='track')['tracks']['items'][0]['external_urls']['spotify']: open_song_url(url))
        button.grid(row=i, column=0, padx=10, pady=5)

    # add a play button to the GUI window


# create GUI elements
instruction_label = Label(root, text='Please select an audio file to predict its emotion:')
instruction_label.pack(pady=20)

select_button = Button(root, text='Select', command=predict_emotion)
select_button.pack(pady=10)

predicted_emotion = Label(root, text='')
predicted_emotion.pack(pady=20)

recommended_songs_frame = Frame(root)
recommended_songs_frame.pack(pady=20)

# start the main event loop for the GUI window
root.mainloop()
